In [4]:
# rSLDS actual data 

In [5]:
IN_COLAB = False
REMOUNT = False

In [6]:
# RUN IN COLAB ONLY: mount Google drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [7]:
# RUN IN COLAB ONLY: unmount, remount Google drive - if required
if REMOUNT:
    !fusermount -u /content/drive 2>/dev/null || true
    !rm -rf /content/drive
    from google.colab import auth
    auth.authenticate_user()
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [8]:
# RUN IN COLAB ONLY: install required modules: ssm, openpyxl
if IN_COLAB:
    !pip -q install git+https://github.com/lindermanlab/ssm.git #egg=ssm
    !pip -q install openpyxl

In [9]:
# RUN IN COLAB ONLY: force-fetch latest files from GitHub (overwrite local copies)
if IN_COLAB:
    !curl -L -H "Cache-Control: no-cache" -o gridsearch.py https://raw.githubusercontent.com/chrismader1/SLDS/main/gridsearch.py
    !curl -L -H "Cache-Control: no-cache" -o rSLDS.py https://raw.githubusercontent.com/chrismader1/SLDS/main/rSLDS.py

In [10]:
# RUN IN COLAB ONLY: prevent timeouts
if IN_COLAB:
    from IPython.display import Javascript, display
    display(Javascript("""
    (function keepAlive(){
      function clickConnect(){
        // Try shadow-root button (newer Colab)
        const el = document.querySelector('colab-connect-button');
        if (el && el.shadowRoot){
          const btn = el.shadowRoot.querySelector('#connect');
          if (btn){ btn.click(); console.log('keepAlive: clicked shadow connect'); return; }
        }
        // Fallbacks
        const btn2 = document.querySelector('#connect, button#connect');
        if (btn2){ btn2.click(); console.log('keepAlive: clicked #connect'); return; }
        console.log('keepAlive: connect button not found');
      }
      setInterval(clickConnect, 60 * 1000);
      console.log('keepAlive: armed');
    })();
    """))

In [11]:
# RUN IN COLAB ONLY: clear cache
if IN_COLAB:
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_results.csv"
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_segments.csv"
    !echo "Reset done."

In [12]:
import numpy as np
import pandas as pd
from gridsearch import pipeline_actual

In [13]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

CONFIG = {
    
    # Core defaults
    "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 50,
    "h_z": 3.0,  # CUSUM parameter
    
    # Batch windows
    "batch_grid": [
        {"train_window": 756, "overlap_window": 5},
        # {"train_window": 256, "overlap_window": 63},
        # {"train_window": 504, "overlap_window": 63},
        # {"train_window": 756, "overlap_window": 63},
        # {"train_window": 1260, "overlap_window": 63},
    ],

    # Number of regimes
    "K_grid": [3, 4],
    
    # Unrestricted models: 
    "unrestricted_models": [
        {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": True,
    "run_restricted": True,

    # Output
    "verbose": False,
    "display": False,

    # IO: Colab/Google Drive
    "data_excel": "/content/drive/MyDrive/SLDS/Data/bbg_data.xlsx",  # Google Drive
    "ff_dir": "/content/drive/MyDrive/SLDS/Data/",                   # Google Drive
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": "/content/drive/MyDrive/SLDS/Out/gridsearch_results.csv",            # Google Drive
    "segments_parquet": "/content/drive/MyDrive/SLDS/Out/gridsearch_segments.parquet",  # Google Drive
    "tmp_dir":          "/content/tmp_slds/",                      # Colab local
    "segments_tmp_csv": "/content/tmp_slds/segments_tmp.csv",      # Colab local

    # IO: Local (macOS)
    # "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    # "ff_dir": "/Users/chrismader/Python/SLDS/Data/",
    # "ff_files": {
    #     "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
    #     "ff3": "F-F_Research_Data_Factors_daily.csv",
    #     "mom": "F-F_Momentum_Factor_daily.csv",},
    # "results_csv": "/Users/chrismader/Python/SLDS/Out/gridsearch_results.csv",
    # "segments_parquet": "/Users/chrismader/Python/SLDS/Out/gridsearch_segments.parquet",
    # "tmp_dir":          "/Users/chrismader/Python/SLDS/tmp_slds/",
    # "segments_tmp_csv": "/Users/chrismader/Python/SLDS/tmp_slds/segments_tmp.csv",
}

In [14]:
# --------------------------------------------------------------------------------------
# Execute
# --------------------------------------------------------------------------------------

def main():
    
    import os
    import pandas as pd

    # IO from CONFIG
    csv_path = CONFIG["results_csv"]
    filename = CONFIG["data_excel"]
    out_path = os.path.dirname(CONFIG["results_csv"]) + "/"
    
    # securities_master = [
    #     "MSFT","NVDA","AAPL","AVGO","GOOGL","ORCL","IBM","CSCO","CRM","AMD","INTU","NOW","TXN","QCOM",
    #     "ADBE","AMAT","PLTR","ACN","META","NFLX","GOOGL","GOOG","DIS","VZ","T","UBER","AMZN","TSLA","HD",
    #     "MCD","BKNG","COST","WMT","PG","KO","PM","PEP","BRK/B","JPM","V","MA","BAC","WFC","GS","MS","SPGI",
    #     "SCHW","AXP","BLK","C","PGR","LLY","JNJ","ABBV","UNH","ABT","MRK","ISRG","BSX","TMO","AMGN","RTX",
    #     "XOM","CVX","GE","CAT","BA","HON","LIN","NEE","DUK","SO","AEP","D","AMT","PLD","EQIX","O","CBRE",]

    securities_master = ["MSFT","NVDA","AAPL"]

    # resume if CSV exists; otherwise start from beginning
    if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
        try:
            df = pd.read_csv(csv_path, usecols=["security"])
            done = set(df["security"].dropna().astype(str))
        except Exception:
            done = set()
        securities = [s for s in securities_master if s not in done]
        print("Resuming from CSV:", csv_path)
        print("Already completed:", sorted(done))
    else:
        securities = securities_master
        print("Resuming from: START (no/empty CSV)")

    print("Remaining tickers:", len(securities), securities)

    if len(securities) == 0:
        print("Nothing to do.")
        return

    _ = pipeline_actual(securities=securities, CONFIG=CONFIG)

if __name__ == "__main__":
    main()


Resuming from: START (no/empty CSV)
Remaining tickers: 3 ['MSFT', 'NVDA', 'AAPL']


MSFT
combo_total= 1


------------------------------------------------------------------------
(1/1) factor2_ff3 | Params: {'n_regimes': 3, 'dim_latent': 3, 'single_subspace': True, 'restrictions': {'C': array([[ 0.01160995, -0.00432058, -0.00362368],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]]), 'd': array([0.00025443, 0.        , 0.        , 0.        ]), 'b_pattern': ['mu_form', 'mu_form', 'mu_form']}, 'model_name': 'factor2_ff3'};{'train_window': 756, 'overlap_window': 5}
------------------------------------------------------------------------

fits succeeded: 5

LEADERBOARD:
 rank   score  n_regimes  dim_latent  single_subspace  model_name
    1 0.00517          3           3             True factor2_ff3
combo_total= 1


------------------------------------------------------------------------
(1/1) f